# Klassifikation von Texten mithilfe von Transformern

*Transformer* sind seit ChatGPT in aller Munde. Mit den "kleinen Geschwistern" von GPT kann man sehr gut Texte klassifizieren und andere NLP-Aufgaben lösen.
Die folgende Beschreibung wie das funktioniert stammt übrigens zu großen Teilen von ChatGPT, lediglich an einigen Stellen habe ich etwas geändert (damit Sie auch noch selbst etwas zu tun haben).

Ja, ich kann Ihnen helfen. Um Twitter-Nachrichten mit einem Transformer-Modell zu klassifizieren, folgen Sie diesen Schritten:

1. Installieren Sie die erforderlichen Bibliotheken:



In [ ]:
%pip install -U transformers

2. Importieren Sie die erforderlichen Bibliotheken:


In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm

3. Implementieren Sie eine benutzerdefinierte Dataset-Klasse

**Aufgabe 1: Ergänzen Sie Code zum Bereinigen der Tweets.**

In [2]:
import re

class GermEvalDataset(Dataset):
    def __init__(self, tokenizer, data_path, max_len):
        self.tokenizer = tokenizer
        self.data = pd.read_csv(data_path, sep='\t', header=None, names=['text', 'label', 'fine'])
        
        self.data['text'] = self.data['text'].apply(self.clean_tweet)

        self.max_len = max_len

    def clean_tweet(self, text):
        """ Preprocess and tokenize a tweet. """
        
        # remove handles, i.e. @username
        text = re.sub('\@\w+', '', text)
    
        # remove hashtags, quotes, etc.
        text = re.sub('[\#"\']+', '', text)
        
        # replace hyphens with blanks
        text = text.replace('-', ' ')
        return text

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.loc[index, 'text']
        label = self.data.loc[index, 'label']
        
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        
        if label == "OTHER":
            label_tensor = torch.tensor(0)
        elif label == "OFFENSE":
            label_tensor = torch.tensor(1)
        else:
            raise ValueError(f"Invalid label: {label} for {text} at {index}")
            
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label_tensor}

4. Laden Sie das pre-trained Modell und den Tokenizer:

**ChatGPT schlägt hier das Modell `"deepset/gbert-large"` vor – eine gute Wahl für deutschsprachige Tweets.
Recherchieren Sie im [Model-Hub von Higging Face](https://huggingface.co/models) ein paar Alternativen und vergleichen Sie die Ergebnisse.**

In [3]:
model_name = "deepset/gbert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5. Erstellen Sie die DataLoader für Training und Validierung:

In [4]:
MAX_LEN = 512
BATCH_SIZE = 16
train_data_path = "../data/GermEval-2018/germeval2018.training.txt"
val_data_path = "../data/GermEval-2018/germeval2018.test.txt"

train_dataset = GermEvalDataset(tokenizer, train_data_path, MAX_LEN)
val_dataset = GermEvalDataset(tokenizer, val_data_path, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [5]:
train_dataset.data.text

0        Liebe Corinna, wir würden dich gerne als Mode...
1        Sie haben ja auch Recht. Unser Tweet war etwa...
2        fröhlicher gruß aus der schönsten stadt der w...
3        Amis hätten alles und jeden gewählt...nur Hil...
4        kein verläßlicher Verhandlungspartner. Nachka...
                              ...                        
5004    Gegens. Zul. zu Patenamt &amp; gegenseitige An...
5005     Zu Merkel fällt mir nur ein, ein Mal Verräter...
5006      Ein richtiges Zeichen unserer Nachbarn...sch...
5007     ,Honecker‘Merkel macht uns zur ,DDR‘ Klagen w...
5008    Warum wurden die G20 Chaoten nicht sofort auf ...
Name: text, Length: 5009, dtype: object

6. Erstellen Sie die Trainings-Argumente:

**Die Vorgaben von ChatGPT sind in Ordnung, aber schauen Sie einmal, was passiert, wenn Sie an den Parametern `BATCH_SIZE` und `learning_rate` "drehen".**

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    report_to=None,
    learning_rate=1e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy="steps",
    seed=42,
)

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


7. Definieren Sie die Metrik-Funktion:

**Hier habe ich geschummelt und die Metriken aus GermEval 2018 "nachgebaut".**

In [7]:
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    accuracy = (preds == p.label_ids).astype(np.float32).mean().item()
    metrics = { "accuracy": accuracy }
    for val, key in enumerate(['OTHER', 'OFFENSE']):
        tp = ((preds == p.label_ids) * (preds == val)).sum().item()
        fp = ((preds != p.label_ids) * (preds == val)).sum().item()
        fn = ((preds != p.label_ids) * (preds != val)).sum().item()

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)
        metrics[f"precision_{key}"] = precision
        metrics[f"recall_{key}"] = recall
        metrics[f"f1_{key}"] = f1
        
    metrics[f"precision_AVERAGE"] = 0.5 * (metrics[f"precision_OTHER"] + metrics[f"precision_OFFENSE"])
    metrics[f"recall_AVERAGE"] = 0.5 * (metrics[f"recall_OTHER"] + metrics[f"recall_OFFENSE"])
    metrics[f"f1_AVERAGE"] = 0.5 * (metrics[f"f1_OTHER"] + metrics[f"f1_OFFENSE"])
    return metrics

8. Erstellen Sie einen Trainer und trainieren Sie das Modell:


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gawron-christian (fhswf) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Precision Other,Recall Other,F1 Other,Precision Offense,Recall Offense,F1 Offense,Precision Average,Recall Average,F1 Average
100,0.503200,0.489590,0.803119,0.788244,0.960409,0.865851,0.864947,0.495652,0.630182,0.826595,0.728031,0.748017


In [ ]:
!wandb logout

9. Optional: Bewerten Sie das Modell nach dem Training:

```python
trainer.evaluate()
```

Das trainierte Modell kann jetzt zur Klassifikation von Twitter-Nachrichten verwendet werden.